# Introduction
blah blah blah
## Next Generation Sequencing (NGS)
blah blah blah
## Methods
1. Download or copy the .fastq files 
2. Download or copy the following programs:
* Cufflinks : [link]
* Bowtie2 : [link]
* etc.
3. Install the programs (don't forget to update your source in terminal with   ``` source ~/.bashrc```   after you have installed the programs.
4. etc.
## Potential problems and pitfalls
blah blah blah
## etc.

# 1. The fastq/fq file format

# 2. Quality control with *FastQC*

# 3. Trimming RNAseq reads with *Trimmomatic*

# 4. Mapping RNAseq reads to the reference genome with *Bowtie2*

# 5. Assembling the mapped reads into a transcriptome with *Cufflinks*

# 6. etc